In [1]:
import gc
import os
import talib
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from os import getenv
from dotenv import load_dotenv

from strategy import long_short
from config import binance_client, prices_collection
from helper import vwap, check_decimals, current_position

load_dotenv()

client = binance_client()
prices = prices_collection()

In [23]:
TOKEN = "YOUR TELEGRAM BOT TOKEN"
url = f"https://api.telegram.org/bot{getenv('TELEGRAM')}/getUpdates"
print(requests.get(url).json())

{'ok': True, 'result': [{'update_id': 864073274, 'message': {'message_id': 1, 'from': {'id': 1860120914, 'is_bot': False, 'first_name': 'Josh', 'last_name': 'Carlisle', 'username': 'jcarlisle', 'language_code': 'en'}, 'chat': {'id': 1860120914, 'first_name': 'Josh', 'last_name': 'Carlisle', 'username': 'jcarlisle', 'type': 'private'}, 'date': 1671716563, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}]}


In [2]:
def telegram_notification(message):
    token = getenv('TELEGRAM')
    chat_id = getenv('TELEGRAM_CHAT_ID')
    message = message

    url = f'https://api.telegram.org/bot{token}/sendMessage?chat_id={chat_id}&text={message}'

    print(requests.get(url).json())

In [3]:
telegram_notification('testtesttest')

{'ok': True, 'result': {'message_id': 2, 'from': {'id': 5586560116, 'is_bot': True, 'first_name': 'carlisle trade bot', 'username': 'carlisle_trade_bot'}, 'chat': {'id': 1860120914, 'first_name': 'Josh', 'last_name': 'Carlisle', 'username': 'jcarlisle', 'type': 'private'}, 'date': 1671717149, 'text': 'testtesttest'}}


In [19]:
last_date = prices.find().sort('date', -1).limit(1).next()['date']
print(f'Finding the last record in our DB')

new_data_query = client.get_historical_klines('ETHUSDT', '5m', str(last_date), limit=100)
print('Getting new data from Binance API (5m interval)')

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Close time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    print(f'Inserting new data into DB')
    prices_collection().insert_many(new_data.to_dict('records'))

df = pd.DataFrame(list(prices.find().sort('date', -1).limit(300)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

print('Calculating signals')
df['signal'] = df.apply(long_short, axis=1)

symbol = 'ETHUSD'
decimal = check_decimals(symbol)

signal = df['signal'].iloc[-1]
price = df['close'].iloc[-1]
position = current_position()
print(f'Current signal: {signal}, current position: {position}')

Finding the last record in our DB
Getting new data from Binance API (5m interval)
Inserting new data into DB
Calculating signals
Current signal: None, current position: SELL


In [20]:
df.tail(20)

,_id,open,high,low,close,vol,rsi,macdhist,EMA9,EMA55,EMA200,date,signal
280,63a3106ffad2977861996042,1213.83,1215.29,1213.83,1215.29,2.70132,53.448200,0.052152,1214.382240,1214.486387,1213.850699,2022-12-21 13:54:59.999,None
281,63a3119afad2977861996045,1215.37,1215.37,1214.36,1214.96,9.36363,51.744677,0.088776,1214.497792,1214.503302,1213.861736,2022-12-21 13:59:59.999,None
282,63a312c6fad2977861996048,1214.77,1214.90,1214.66,1214.84,4.17209,51.106787,0.101427,1214.566234,1214.515327,1213.871470,2022-12-21 14:04:59.999,BUY
283,63a31da6c59dfd87b1c9ecda,1215.12,1215.16,1214.49,1215.12,61.12322,52.575852,0.123684,1214.676987,1214.536922,1213.883893,2022-12-21 14:09:59.999,BUY
284,63a31da6c59dfd87b1c9ecdb,1214.75,1215.01,1214.31,1214.31,1.33733,48.075677,0.080951,1214.603590,1214.528818,1213.888133,2022-12-21 14:14:59.999,None
285,63a31da6c59dfd87b1c9ecdc,1214.71,1215.02,1214.71,1214.84,1.18055,51.029299,0.085056,1214.650872,1214.539931,1213.897605,2022-12-21 14:19:59.999,BUY
286,63a31da6c59dfd87b1c9ecdd,1214.74,1215.57,1214.30,1215.16,5.74980,52.775950,0.104708,1214.752697,1214.562077,1213.910166,2022-12-21 14:24:59.999,BUY
287,63a31da6c59dfd87b1c9ecde,1214.69,1216.12,1214.34,1215.62,1.84186,55.247016,0.141769,1214.926158,1214.599860,1213.927179,2022-12-21 14:29:59.999,BUY
288,63a31da6c59dfd87b1c9ecdf,1215.61,1217.01,1212.04,1212.04,13.64666,38.404334,-0.070348,1214.348926,1214.508436,1213.908401,2022-12-21 14:34:59.999,SELL
289,63a31da6c59dfd87b1c9ece0,1212.52,1216.38,1211.95,1215.92,16.35747,54.569587,0.048716,1214.663141,1214.558849,1213.928417,2022-12-21 14:39:59.999,BUY


In [ ]:
signal = df['signal'].iloc[-1]
position = current_position()

if signal == position:
    print('No change in position')
    
else:
    with open('current_position.txt', 'w') as f:
        f.write(signal)

    if signal == 'BUY':
        print(f'Signal went from: {position} to {signal}, flipping to long at {df["close"].iloc[-1]}')
        balance = client.get_asset_balance(asset='USD')
        quantity = round((float(balance['free']) * .95), decimal)
        order = client.create_order(symbol=symbol, side=signal, type='MARKET', quantity=quantity)
        print(order)

    if signal == 'SELL':
        print(f'Signal went from: {position} to {signal}, flipping to short at {df["close"].iloc[-1]}')
        balance = client.get_asset_balance(asset='ETH')
        quantity = round((float(balance['free']) * .95), decimal)
        order = client.create_order(symbol=symbol, side=signal, type='MARKET', quantity=quantity)
        print(order)

    if signal == 'None':
        print('No signal, holding position')
        pass

In [ ]:
df.to_csv('check.csv', index=False)

In [ ]:
df.plot(y=['close', 'EMA20'], x='date', figsize=(20, 10))